# Objectives

- Understand the workings of an algorithmic stablecoin - DAI
- Making a CDP vault 
- Opening up a proxy wallet contract and setting up a stop-loss service
- High-level overview of automated strategies

# Prerequisites 

- Metamask 
- Ensure a minimum of 0.2 GETH in your account
- (Optional) 3.5 GETH or more to mint DAI


# Opening collateral vault on Oasis Web-app

DAI is one of the prominent algorithmic stablecoins in the Ethereum ecosystem; it derives its stability using the leveraged loan primitive using the MakerDAO system. Minters need to open up a vault and deposit collateral to mint DAI. This position represents a collateralized debt position whose security/peg relies on liquidating vaults in case of significant price deviations. In this lab, we will open up collateral vaults in Ethereum, set up automatic vault management using proxy wallets, and see how to set up stop-loss protections using the proxy wallet on the web app that interacts with the MakerDAO contract - Oasis.

## Opening a vault in Oasis 

- Go to Oasis borrow at [link](https://oasis.app/borrow?network=goerli)
- Sign the message using metamask to accept their terms and conditions (this is a new practice in web apps that manage funds more actively)
- The interface looks similar to a borrow-lending platform with one significant difference - different terms for the same type of collateral 
- Collateralizing ETH using different types of vaults render different returns and accepts different collateralization ratios 
- Each vault sets a minimum amount of DAI that can be minted - a debt floor - the smallest vault sets this limit as 3500 DAI
- Scan through different types of vaults, and observe the inverse relation between DAI debt floor limit and collateralization ratio - why does such an inverse relation exist? - To facilitate liquidation profit in high tx-fee scenarios
- We may not have enough ETH to mint DAI; however, we can still create vaults to observe some other novel features of the app
- How is using collateral to mint DAI different than borrowing DAI using Aave?
- Click on ETH-C vault; this is the vault with the lowest debt floor one can create
- We will demonstrate creating the vault by depositing a small amount of ETH (0.01 ETH)

### Setting up a proxy 

- Note that if you are opening the vault for the first time on Maker, it will ask you to create a proxy wallet contract - this is often referred to as a smart wallet 
- The proxy contract acts on behalf of the wallet to perform more complex actions and also allows a wallet to outsource some pre-programmed actions to an automated system
- The smart wallet performs actions prescribed by its owner if its call for "execute" is signed by the owner. Any authorized service can call `execute` as long as it's signed by the owner.
- Observe the proxy creation transaction [example here](https://goerli.etherscan.io/tx/0x6ba2a2b5d97f9f8c3742be38201157fb4d4426811c9d136d7631379a308bd9d4). The transaction uses the build() function to create the proxy wallet contract that your wallet will use to perform transactions with Maker [contract code](https://goerli.etherscan.io/address/0x46759093d8158db8bb555ac7c6f98070c56169ce#code)
- Once the proxy is created, click on confirm to create the vault [example tx](https://goerli.etherscan.io/tx/0x0d3f8401029b56906a36ae8ff00d886990ed7fd1dd6ceac4d68ca85f7327c85c). Notice the transfer of ETH and DAI (note that the DAI is minted, not transferred form a contract)

### Setting up stop-loss 
Utilizing this service was why a proxy was created. Post the `execute` transaction [example](https://goerli.etherscan.io/tx/0xff68672ee64ec42df1b0ded06cc32ceb348f82394e254d6b901f4c3265fdcc03) to set up the service. The stop-loss service will be automatically triggered when the ETH price reaches a threshold set by the user (defaults to 180% collateralization ratio). MakerDAO maintains a bot that monitors ETH prices on an hourly basis using the chainlink oracle; when the price crosses the user's threshold, it broadcasts a transaction that was pre-signed by the user that performs the following steps: 
- Flash mint DAI
- Pay back Debt
- Withdraw collateral
- Sell collateral for DAI
- Pay the flash-minted DAI back
This automated process ensures that the vault owner does not have to be active all the time during the vault creation process. 

You can get a more detailed look into automated services using the [DefiSaver](https://app.defisaver.com/) platform